# Collecting Data 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
import requests
import io

In [ ]:
# Start the chrome browser
s=Service(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(service=s, chrome_options=chrome_options)
driver.maximize_window()

In [ ]:
df=pd.read_csv('zip_code_database.csv')
ca_df=df.loc[df.state=='CA',]
county_list=list(set(ca_df.county))
import numpy as np
county_list.remove(np.nan)
print(f"There are {len(ca_df)} unique zip codes and {len(county_list)} unique counties in CA.\n")
print(county_list)

In [ ]:
def searchBycounty(county_name):
    driver.get('https://www.redfin.com')
    time.sleep(3)
    try:
        driver.find_element(by=By.XPATH,value="//input[contains(@id, 'search-box-input') and contains(@title, 'City, Address, School, Agent, ZIP')]").click()
        driver.find_element(by=By.XPATH,value="//input[contains(@id, 'search-box-input') and contains(@placeholder, 'City, Address, School, Agent, ZIP')]").send_keys(county_name)
        time.sleep(3)
        driver.find_element(by=By.XPATH,value="//a[text()='" + county_name + "']").click()
        download_link = driver.find_element(by=By.XPATH,value="//a[contains(@id, 'download-and-save')]").get_attribute("href")
        all_links = download_link.split('num_homes=350')
        return (all_links[0]+'num_homes=10000'+all_links[1])
    except Exception as e:
        return ('No link available')

In [ ]:
df_link_county=pd.DataFrame()
for i in range(len(county_list)):
    df_link_county.loc[i,'county']=county_list[i]
    tmp=searchBycounty(county_list[i])
    df_link_county.loc[i,'link']=tmp
driver.close()

In [ ]:
df_link_county.to_csv('county_list_link.csv',index=False)

In [3]:
df_link_county=pd.read_csv('county_list_link.csv')

In [10]:
url = df_link_county.link[0]
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
result = requests.get(url, headers=headers)
data=result.content.decode()
type(result.content)
# df_trial = pd.read_csv(io.StringIO(data))
# df_trial

bytes

In [ ]:
df_detail=pd.DataFrame()
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
for i in range(len(df_link_county)):
    if df_link_county.link[i]!='No link available':
        url=df_link_county.link[i]
        results=pd.read_csv(io.StringIO(requests.get(url,headers=headers).content.decode()))
        df_detail=df_detail.append(results, ignore_index = True)

In [11]:
df_detail.to_csv('housing detail.csv',index=False)

In [12]:
df_detail=pd.read_csv('housing detail.csv')
df_detail.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME',
       'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)',
       'SOURCE', 'MLS#', 'FAVORITE', 'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

# url paragraph 

In [25]:
column_list=[i.lower() for i in df_detail.columns.to_list()]
column_list[20]='url'
df_detail.columns=column_list

In [108]:
s=Service(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(service=s, chrome_options=chrome_options)
driver.maximize_window()



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/georgiaxiao/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache
<ipython-input-108-dc55e06ceddb>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=s, chrome_options=chrome_options)


In [147]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException

In [111]:
for i in range(len(df_detail)):
    url_toget = df_detail.loc[i,'url']
    try:
        driver.get(url_toget)
    except WebDriverException:
        s=Service(ChromeDriverManager().install())
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        driver = webdriver.Chrome(service=s, chrome_options=chrome_options)
        driver.maximize_window()
        driver.get(url_toget)
        time.sleep(5)
    else:
        time.sleep(5)
    try:
        driver.find_element(by=By.XPATH,value="//div[contains(@data-react-server-root-id,'22')]/.//span[contains(@class, 'bottomLink font-color-link clickable')]")
    except NoSuchElementException:
        try:
            driver.find_element(by=By.XPATH,value="//div[contains(@class,'remarks') and contains(@id,'marketing-remarks-scroll')]/p[contains(@class,'text-base')]/span").get_attribute('innerHTML')
        except NoSuchElementException:
            a='no remarks'
        else:
            a=driver.find_element(by=By.XPATH,value="//div[contains(@class,'remarks') and contains(@id,'marketing-remarks-scroll')]/p[contains(@class,'text-base')]/span").get_attribute('innerHTML')
    else:
        driver.find_element(by=By.XPATH,value="//div[contains(@data-react-server-root-id,'22')]/.//span[contains(@class, 'bottomLink font-color-link clickable')]").click()
        a=driver.find_element(by=By.XPATH,value="//div[contains(@class,'remarks') and contains(@id,'marketing-remarks-scroll')]/p[contains(@class,'text-base')]/span").get_attribute('innerHTML')
    df_detail.loc[i,'remarks']=a

WebDriverException: Message: unknown error: cannot determine loading status
from disconnected: Unable to receive message from renderer
  (Session info: chrome=101.0.4951.54)
Stacktrace:
0   chromedriver                        0x000000010331e2c9 chromedriver + 5120713
1   chromedriver                        0x00000001032ace33 chromedriver + 4656691
2   chromedriver                        0x0000000102e9c158 chromedriver + 393560
3   chromedriver                        0x0000000102e86fa2 chromedriver + 307106
4   chromedriver                        0x0000000102e85efa chromedriver + 302842
5   chromedriver                        0x0000000102e863fc chromedriver + 304124
6   chromedriver                        0x0000000102e95af5 chromedriver + 367349
7   chromedriver                        0x0000000102e8781d chromedriver + 309277
8   chromedriver                        0x0000000102e885cd chromedriver + 312781
9   chromedriver                        0x0000000102e87a87 chromedriver + 309895
10

In [137]:
df_detail

,sale type,sold date,property type,address,city,state or province,zip or postal code,price,beds,baths,...,next open house start time,next open house end time,url,source,mls#,favorite,interested,latitude,longitude,remarks
0,MLS Listing,NaN,Single Family Residential,907 Boccoli St,Sonoma,CA,95476,1075000.0,3.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Sonoma/907-Boccoli-S...,BAREIS,322036282,N,Y,38.284712,-122.461696,Minutes to Sonoma Plaza! Move in ready Ledson ...
1,MLS Listing,NaN,Condo/Co-op,316 Gate Way,Santa Rosa,CA,95401,398000.0,2.0,1.5,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/316-Gate-...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40989873,N,Y,38.431629,-122.744726,"Come see this lovely move-in ready end unit, t..."
2,MLS Listing,NaN,Single Family Residential,2401 Donna Maria Way,Santa Rosa,CA,95401,649000.0,3.0,2.0,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/2401-Donn...,BAREIS,322031711,N,Y,38.435883,-122.764649,Super clean move in ready home in a quiet neig...
3,MLS Listing,NaN,Condo/Co-op,1702 Glenbrook Dr Unit E,Santa Rosa,CA,95401,399000.0,2.0,1.5,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/1702-Glen...,BAREIS,322031149,N,Y,38.436688,-122.743065,"This 2-story, 2-bedroom, 1.5 bath condo is imm..."
4,MLS Listing,NaN,Condo/Co-op,2514 Westberry Dr,Santa Rosa,CA,95403,399000.0,2.0,1.0,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/2514-West...,BAREIS,322031170,N,Y,38.455801,-122.737994,"This beautifully remodeled 2 bedroom, 1 bath c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41741,New Construction Plan,NaN,Single Family Residential,Plan 7,Temecula,CA,92591,769000.0,2.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Temecula/Esplanade-a...,NewHomeSource.com,Plan-181882631,N,Y,33.544552,-117.102692,NaN
41742,New Construction Plan,NaN,Single Family Residential,Julia,Menifee,CA,92587,706990.0,3.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Menifee/Noble-at-Aud...,NewHomeSource.com,Plan-181924910,N,Y,33.690893,-117.230464,NaN
41743,New Construction Plan,NaN,Single Family Residential,Paige,Lake Elsinore,CA,92530,628990.0,3.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Lake-Elsinore/Runnin...,NewHomeSource.com,Plan-181939330,N,Y,33.698885,-117.402011,NaN
41744,New Construction Plan,NaN,Single Family Residential,Residence One,Indio,CA,92203,495865.0,3.0,2.0,...,NaN,NaN,https://www.redfin.com/CA/Indio/Espana-Almeria...,NewHomeSource.com,Plan-181902300,N,Y,33.762245,-116.281966,NaN


In [145]:
zancun.to_csv('retail with remark_589.csv',index=False)

In [138]:
zancun=df_detail.loc[0:589,].copy()
df2=df_detail.loc[590:,].copy().reset_index()
df2

,index,sale type,sold date,property type,address,city,state or province,zip or postal code,price,beds,...,next open house start time,next open house end time,url,source,mls#,favorite,interested,latitude,longitude,remarks
0,590,MLS Listing,NaN,Vacant Land,0 Crane Canyon Rd,Santa Rosa,CA,95405,515000.0,NaN,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/Crane-Can...,BAREIS,321099959,N,Y,38.367382,-122.652726,NaN
1,591,MLS Listing,NaN,Vacant Land,0 120 Big Leaf Ct,Healdsburg,CA,95448,1795000.0,NaN,...,NaN,NaN,https://www.redfin.com/CA/Healdsburg/120-Big-L...,San Francisco MLS,321099810,N,Y,38.610466,-122.869161,NaN
2,592,MLS Listing,NaN,Vacant Land,22202 Umland Cir,Jenner,CA,95450,249000.0,NaN,...,NaN,NaN,https://www.redfin.com/CA/Jenner/22202-Umland-...,BAREIS,321099785,N,Y,38.541155,-123.278734,NaN
3,593,MLS Listing,NaN,Vacant Land,11025 Occidental Rd,Sebastopol,CA,95472,195000.0,NaN,...,NaN,NaN,https://www.redfin.com/CA/Sebastopol/11025-Occ...,BAREIS,321091566,N,Y,38.407635,-122.892071,NaN
4,594,MLS Listing,NaN,Multi-Family (5+ Unit),80 Front St,Healdsburg,CA,95448,2950000.0,8.0,...,NaN,NaN,https://www.redfin.com/CA/Healdsburg/80-Front-...,BAREIS,321096095,N,Y,38.607038,-122.861704,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41151,41741,New Construction Plan,NaN,Single Family Residential,Plan 7,Temecula,CA,92591,769000.0,2.0,...,NaN,NaN,https://www.redfin.com/CA/Temecula/Esplanade-a...,NewHomeSource.com,Plan-181882631,N,Y,33.544552,-117.102692,NaN
41152,41742,New Construction Plan,NaN,Single Family Residential,Julia,Menifee,CA,92587,706990.0,3.0,...,NaN,NaN,https://www.redfin.com/CA/Menifee/Noble-at-Aud...,NewHomeSource.com,Plan-181924910,N,Y,33.690893,-117.230464,NaN
41153,41743,New Construction Plan,NaN,Single Family Residential,Paige,Lake Elsinore,CA,92530,628990.0,3.0,...,NaN,NaN,https://www.redfin.com/CA/Lake-Elsinore/Runnin...,NewHomeSource.com,Plan-181939330,N,Y,33.698885,-117.402011,NaN
41154,41744,New Construction Plan,NaN,Single Family Residential,Residence One,Indio,CA,92203,495865.0,3.0,...,NaN,NaN,https://www.redfin.com/CA/Indio/Espana-Almeria...,NewHomeSource.com,Plan-181902300,N,Y,33.762245,-116.281966,NaN


In [357]:
s=Service(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(service=s, chrome_options=chrome_options)
driver.maximize_window()
for i in range(len(df4)):
    url_toget = df4.loc[i,'url']
    try:
        driver.get(url_toget)
    except WebDriverException:
        s=Service(ChromeDriverManager().install())
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument("--incognito")
        driver = webdriver.Chrome(service=s, chrome_options=chrome_options)
        driver.maximize_window()
        driver.get(url_toget)
        time.sleep(5)
    else:
        time.sleep(5)
    try:
        driver.find_element(by=By.XPATH,value="//div[contains(@data-react-server-root-id,'22')]/.//span[contains(@class, 'bottomLink font-color-link clickable')]")
    except NoSuchElementException:
        try:
            driver.find_element(by=By.XPATH,value="//div[contains(@class,'remarks') and contains(@id,'marketing-remarks-scroll')]/p[contains(@class,'text-base')]/span").get_attribute('innerHTML')
        except NoSuchElementException:
            a='no remarks'
        else:
            a=driver.find_element(by=By.XPATH,value="//div[contains(@class,'remarks') and contains(@id,'marketing-remarks-scroll')]/p[contains(@class,'text-base')]/span").get_attribute('innerHTML')
    else:
        driver.find_element(by=By.XPATH,value="//div[contains(@data-react-server-root-id,'22')]/.//span[contains(@class, 'bottomLink font-color-link clickable')]").click()
        a=driver.find_element(by=By.XPATH,value="//div[contains(@class,'remarks') and contains(@id,'marketing-remarks-scroll')]/p[contains(@class,'text-base')]/span").get_attribute('innerHTML')
    df4.loc[i,'remarks']=a



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Trying to download new driver from https://chromedriver.storage.googleapis.com/101.0.4951.41/chromedriver_mac64.zip
Driver has been saved in cache [/Users/georgiaxiao/.wdm/drivers/chromedriver/mac64/101.0.4951.41]
<ipython-input-357-0ba1dcad3c82>:4: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=s, chrome_options=chrome_options)


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/georgiaxiao/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache
<ipython-input-357-0ba1dcad3c82>:14: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=s, chrome_options=chrome_options)


====== WebDriver manager ======
Current google-chrome version is 101.0.4951



====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/georgiaxiao/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/georgiaxiao/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/georgiaxiao/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


====== WebDriver manager ======
Current google-chrome version is 101.0.4951
Get LATEST chromedriver version for 101.0.4951 google-chrome
Driver [/Users/georgiaxiao/.wdm/drivers/chromedriver/mac64/101.0.4951.41/chromedriver] found in cache


====== WebDriver manager ======
Cu

In [333]:
driver.close()

In [331]:
zancun5=pd.concat([zancun4,a.loc[0:9691,]],axis=0)
zancun5.to_csv('detail remarks 25156.csv',index=False)
df4=a.iloc[9692:,].reset_index(drop=True).copy()

In [356]:
# df4.iloc[0:2867,]
# zancun5
zancun6=pd.concat([zancun5,df4.iloc[0:2867,]],axis=0)
zancun6.to_csv('detail remarks 28023.csv',index=False)
df4=df4.iloc[2868:,].reset_index(drop=True).copy()

In [302]:
left_df4=df_detail.loc[15465:,['sale type','sold date','property type','address','city','state or province','zip or postal code']].reset_index(drop=True)
a=pd.concat([left_df4,df4],axis=1)

True

In [330]:
a=pd.concat([left_df4,df4],axis=1)

a.iloc[9692:,].reset_index(drop=True).copy()

,sale type,sold date,property type,address,city,state or province,zip or postal code,price,beds,baths,...,next open house start time,next open house end time,url,source,mls#,favorite,interested,latitude,longitude,remarks
0,Redfin Data,NaN,Single Family Residential,22784 Prime Ave,Hayward,CA,94541,899000.0,3.0,2.0,...,NaN,NaN,https://www.redfin.com/CA/Hayward/22784-Prime-...,Coming Soon,83333,N,Y,37.685540,-122.053004,NaN
1,Redfin Data,NaN,Single Family Residential,27072 Belfast Ln,Hayward,CA,94542,1325000.0,4.0,3.0,...,NaN,NaN,https://www.redfin.com/CA/Hayward/27072-Belfas...,Coming Soon,83287,N,Y,37.655567,-122.042260,NaN
2,MLS Listing,NaN,Condo/Co-op,4053 Harlan St #302,Emeryville,CA,94608,650000.0,1.0,1.0,...,NaN,NaN,https://www.redfin.com/CA/Emeryville/4053-Harl...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40990273,N,Y,37.830556,-122.283904,NaN
3,MLS Listing,NaN,Multi-Family (2-4 Unit),1455 - 1453 84th Ave,Oakland,CA,94621,650000.0,3.0,2.0,...,NaN,NaN,https://www.redfin.com/CA/Oakland/1455-84th-Av...,MetroList,222050560,N,Y,37.755176,-122.176845,NaN
4,MLS Listing,NaN,Single Family Residential,3941 Ardley Ave,Oakland,CA,94602,995000.0,2.0,1.5,...,April-30-2022 02:00 PM,April-30-2022 04:00 PM,https://www.redfin.com/CA/Oakland/3941-Ardley-...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40990300,N,Y,37.804833,-122.223176,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16584,New Construction Plan,NaN,Single Family Residential,Plan 7,Temecula,CA,92591,769000.0,2.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Temecula/Esplanade-a...,NewHomeSource.com,Plan-181882631,N,Y,33.544552,-117.102692,NaN
16585,New Construction Plan,NaN,Single Family Residential,Julia,Menifee,CA,92587,706990.0,3.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Menifee/Noble-at-Aud...,NewHomeSource.com,Plan-181924910,N,Y,33.690893,-117.230464,NaN
16586,New Construction Plan,NaN,Single Family Residential,Paige,Lake Elsinore,CA,92530,628990.0,3.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Lake-Elsinore/Runnin...,NewHomeSource.com,Plan-181939330,N,Y,33.698885,-117.402011,NaN
16587,New Construction Plan,NaN,Single Family Residential,Residence One,Indio,CA,92203,495865.0,3.0,2.0,...,NaN,NaN,https://www.redfin.com/CA/Indio/Espana-Almeria...,NewHomeSource.com,Plan-181902300,N,Y,33.762245,-116.281966,NaN


In [364]:
df_detail_remarks=pd.concat([zancun6,df4],axis=0)
df_detail_remarks.to_csv('detial_remarks_full_41744.csv',index=False)

In [365]:
driver.close()

In [11]:
df_detail_remarks=pd.read_csv('detial_remarks_full_41744.csv')
df_detail_remarks

,sale type,sold date,property type,address,city,state or province,zip or postal code,price,beds,baths,...,next open house start time,next open house end time,url,source,mls#,favorite,interested,latitude,longitude,remarks
0,MLS Listing,NaN,Single Family Residential,907 Boccoli St,Sonoma,CA,95476,1075000.0,3.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Sonoma/907-Boccoli-S...,BAREIS,322036282,N,Y,38.284712,-122.461696,Minutes to Sonoma Plaza! Move in ready Ledson ...
1,MLS Listing,NaN,Condo/Co-op,316 Gate Way,Santa Rosa,CA,95401,398000.0,2.0,1.5,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/316-Gate-...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40989873,N,Y,38.431629,-122.744726,"Come see this lovely move-in ready end unit, t..."
2,MLS Listing,NaN,Single Family Residential,2401 Donna Maria Way,Santa Rosa,CA,95401,649000.0,3.0,2.0,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/2401-Donn...,BAREIS,322031711,N,Y,38.435883,-122.764649,Super clean move in ready home in a quiet neig...
3,MLS Listing,NaN,Condo/Co-op,1702 Glenbrook Dr Unit E,Santa Rosa,CA,95401,399000.0,2.0,1.5,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/1702-Glen...,BAREIS,322031149,N,Y,38.436688,-122.743065,"This 2-story, 2-bedroom, 1.5 bath condo is imm..."
4,MLS Listing,NaN,Condo/Co-op,2514 Westberry Dr,Santa Rosa,CA,95403,399000.0,2.0,1.0,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/2514-West...,BAREIS,322031170,N,Y,38.455801,-122.737994,"This beautifully remodeled 2 bedroom, 1 bath c..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41739,New Construction Plan,NaN,Single Family Residential,Plan 7,Temecula,CA,92591,769000.0,2.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Temecula/Esplanade-a...,NewHomeSource.com,Plan-181882631,N,Y,33.544552,-117.102692,Located in 55+ resort-lifestyle community Espl...
41740,New Construction Plan,NaN,Single Family Residential,Julia,Menifee,CA,92587,706990.0,3.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Menifee/Noble-at-Aud...,NewHomeSource.com,Plan-181924910,N,Y,33.690893,-117.230464,Exceptional design and abundant options make t...
41741,New Construction Plan,NaN,Single Family Residential,Paige,Lake Elsinore,CA,92530,628990.0,3.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Lake-Elsinore/Runnin...,NewHomeSource.com,Plan-181939330,N,Y,33.698885,-117.402011,The spacious Paige plan offers ample room for ...
41742,New Construction Plan,NaN,Single Family Residential,Residence One,Indio,CA,92203,495865.0,3.0,2.0,...,NaN,NaN,https://www.redfin.com/CA/Indio/Espana-Almeria...,NewHomeSource.com,Plan-181902300,N,Y,33.762245,-116.281966,This cozy single-story home maximizes space wi...


In [29]:
df_detail_remarks['property type'].unique()

array(['Single Family Residential', 'Condo/Co-op', 'Vacant Land',
       'Mobile/Manufactured Home', 'Townhouse', 'Multi-Family (2-4 Unit)',
       'Multi-Family (5+ Unit)', 'Timeshare', 'Other', 'Ranch', 'Unknown'],
      dtype=object)

## keywords 

In [3]:
df=pd.read_csv("detial_remarks_full_41744.csv")
df.head()

,sale type,sold date,property type,address,city,state or province,zip or postal code,price,beds,baths,...,next open house start time,next open house end time,url,source,mls#,favorite,interested,latitude,longitude,remarks
0,MLS Listing,NaN,Single Family Residential,907 Boccoli St,Sonoma,CA,95476,1075000.0,3.0,2.5,...,NaN,NaN,https://www.redfin.com/CA/Sonoma/907-Boccoli-S...,BAREIS,322036282,N,Y,38.284712,-122.461696,Minutes to Sonoma Plaza! Move in ready Ledson ...
1,MLS Listing,NaN,Condo/Co-op,316 Gate Way,Santa Rosa,CA,95401,398000.0,2.0,1.5,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/316-Gate-...,"bridgeMLS, Bay East AOR, or Contra Costa AOR",40989873,N,Y,38.431629,-122.744726,"Come see this lovely move-in ready end unit, t..."
2,MLS Listing,NaN,Single Family Residential,2401 Donna Maria Way,Santa Rosa,CA,95401,649000.0,3.0,2.0,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/2401-Donn...,BAREIS,322031711,N,Y,38.435883,-122.764649,Super clean move in ready home in a quiet neig...
3,MLS Listing,NaN,Condo/Co-op,1702 Glenbrook Dr Unit E,Santa Rosa,CA,95401,399000.0,2.0,1.5,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/1702-Glen...,BAREIS,322031149,N,Y,38.436688,-122.743065,"This 2-story, 2-bedroom, 1.5 bath condo is imm..."
4,MLS Listing,NaN,Condo/Co-op,2514 Westberry Dr,Santa Rosa,CA,95403,399000.0,2.0,1.0,...,NaN,NaN,https://www.redfin.com/CA/Santa-Rosa/2514-West...,BAREIS,322031170,N,Y,38.455801,-122.737994,"This beautifully remodeled 2 bedroom, 1 bath c..."


In [5]:
keyword_list=df.columns.tolist()
keyword_list

['sale type',
 'sold date',
 'property type',
 'address',
 'city',
 'state or province',
 'zip or postal code',
 'price',
 'beds',
 'baths',
 'location',
 'square feet',
 'lot size',
 'year built',
 'days on market',
 '$/square feet',
 'hoa/month',
 'status',
 'next open house start time',
 'next open house end time',
 'url',
 'source',
 'mls#',
 'favorite',
 'interested',
 'latitude',
 'longitude',
 'remarks']

### The distance methods to be used:
 - Damerau-Levenshtein Distance
 - Jaro-Winkler Distance

In [ ]:
import spacy
nlp = spacy.load('en_core_web_md')
# len(nlp.vocab)

# Bert Analysis 

## trial with zillow 

In [ ]:
#try with zillow
s=Service(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("--incognito")
driver = webdriver.Chrome(service=s, chrome_options=chrome_options)
driver.maximize_window()
driver.get('https://www.zillow.com')
time.sleep(30)
driver.find_element(By.XPATH,"//input[contains(@placeholder,'Enter an address, neighborhood, city, or ZIP code') and contains(@id, 'search-box-input')]").click()
if driver.find_element(By.XPATH,"//input[contains(@placeholder,'Enter an address, neighborhood, city, or ZIP code') and contains(@id, 'search-box-input')]").is_displayed():
    driver.find_element(By.XPATH,"//input[contains(@placeholder,'Enter an address, neighborhood, city, or ZIP code') and contains(@id, 'search-box-input')]").click()
time.sleep(3)
countyName='Santa Cruz County'
driver.find_element(By.XPATH, "//input[contains(@id, 'search-box-input') and contains(@placeholder, 'Enter an address, neighborhood, city, or ZIP code')]").send_keys(countyName)
time.sleep(3)
driver.find_element(By.XPATH, "//button[contains(@id, 'search-icon')]").click()
time.sleep(3)
driver.find_element(By.XPATH,"//button[contains(@class, 'StyledButton-c11n-8-63-0__sc-wpcbcc-0 iIzpwS')][1]").click()

In [ ]:
driver.close()

In [ ]:
session=requests.session()
response=session.get('https://www.zillow.com/homes/for_rent/santacruz/',headers={'accept':'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
                                                                                'accept-encoding':'gzip, deflate, br',
                                                                                'accept-language':'en-US,en;q=0.8',
                                                                                'upgrade-insecure-requests':'1',
                                                                                'user-agent':'Mozilla/5.0 (Window NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'})

In [ ]:
soup=BeautifulSoup(response.content)

In [ ]:
soup.find_all(class_='list-card-link')

# try zillow with medium

In [ ]:
req_headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'en-US,en;q=0.8',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/61.0.3163.100 Safari/537.36'
}
with requests.Session() as s:
   city = 'seattle/' #*****change this city to what you want*****
   url = 'https://www.zillow.com/homes/for_sale/'+city    
   r = s.get(url, headers=req_headers)
#add contents of urls to soup variable from each url
soup = BeautifulSoup(r.content, 'html.parser')
# soup1 = BeautifulSoup(r2.content, 'html.parser')
# soup2 = BeautifulSoup(r3.content, 'html.parser')
# soup3 = BeautifulSoup(r4.content, 'html.parser')
# soup4 = BeautifulSoup(r5.content, 'html.parser')
# soup5 = BeautifulSoup(r6.content, 'html.parser')
# soup6 = BeautifulSoup(r7.content, 'html.parser')
# soup7 = BeautifulSoup(r8.content, 'html.parser')
# soup8 = BeautifulSoup(r9.content, 'html.parser')
# soup9 = BeautifulSoup(r10.content, 'html.parser') 
soup

In [ ]:
soup.find_all("ul", class_="list-card-details")

In [ ]:
#create the first two dataframes
df = pd.DataFrame()
df1 = pd.DataFrame()
#all for loops are pulling the specified variable using beautiful soup and inserting into said variable
for i in soup:
    address = soup.find_all (class_= 'list-card-addr')
    price = list(soup.find_all (class_='list-card-price'))
    beds = list(soup.find_all("ul", class_="list-card-details"))
    details = soup.find_all ('div', {'class': 'list-card-details'})
    home_type = soup.find_all ('div', {'class': 'list-card-footer'})
    last_updated = soup.find_all ('div', {'class': 'list-card-top'})
    brokerage = list(soup.find_all(class_= 'list-card-brokerage list-card-img-overlay',text=True))
    link = soup.find_all (class_= 'list-card-link')
    
    #create dataframe columns out of variables
    df['prices'] = price
    df['address'] = address
    df['beds'] = beds
#create empty url list
urls = []
#loop through url, pull the href and strip out the address tag
for link in soup.find_all("article"):
    href = link.find('a',class_="list-card-link")
    addresses = href.find('address')
    addresses.extract()
    urls.append(href)
#import urls into a links column
df['links'] = urls
df['links'] = df['links'].astype('str')
#remove html tags
df['links'] = df['links'].replace('<a class="list-card-link" href="', ' ', regex=True)
df['links'] = df['links'].replace('" tabindex="0"></a>', ' ', regex=True)

In [ ]:
df=pd.read_csv('')